In [3]:
import sys
sys.path.append('../')

%matplotlib inline

# DEFINE GRAPH
import pandas as pd
import numpy as np
import keras

from keras.utils import np_utils
from keras import backend as K
from utils import ArgsProxy
import utils

from importlib import reload
reload(utils)

pd.set_option('max_colwidth', 800)

from helpers import process_image

In [4]:
# Data paths
rgb_path = '../data/jpg_128_128_2fps/'
of_path = '../data/jpg_128_128_16fps_OF_magnitude_cv2/'

# Hyperparameters
input_width = 128
input_height = 128
seq_length = 10
seq_stride = 10
batch_size = 1
COLOR = True
nb_labels = 2

# Data augmentation
aug_flip = 0
aug_crop = 0
aug_light = 0

nb_input_dims =5

subject_ids = pd.read_csv('../metadata/horse_subjects.csv')['Subject'].values
channels = 3


args = ArgsProxy(rgb_path, of_path, input_height, input_width,
                 seq_length, seq_stride, batch_size, nb_labels,
                 aug_flip, aug_crop, aug_light, nb_input_dims)

In [5]:
import data_handler as dathand
dh = dathand.DataHandler(path=args.data_path,
                         of_path=args.of_path,
                         clip_list_file='videos_overview_missingremoved.csv',
                         data_columns=['Pain'],  # Here one can append f. ex. 'Observer'
                         image_size=(args.input_height, args.input_width),
                         seq_length=args.seq_length,
                         seq_stride=args.seq_stride,
                         batch_size=args.batch_size,
                         color=COLOR,
                         nb_labels=args.nb_labels,
                         aug_flip=args.aug_flip,
                         aug_crop=args.aug_crop,
                         aug_light=args.aug_light,
                         nb_input_dims=args.nb_input_dims)

In [6]:
import utils
subject_dfs = utils.read_or_create_subject_dfs(dh, args, subject_ids)

../data/jpg_128_128_2fps/
../data/jpg_128_128_2fps/
../data/jpg_128_128_2fps/
../data/jpg_128_128_2fps/
../data/jpg_128_128_2fps/
../data/jpg_128_128_2fps/


In [7]:
subject_dfs = utils.read_or_create_subject_rgb_and_OF_dfs(dh, args, subject_ids, subject_dfs)

## Get an example sequence (random from fifth subject)

In [8]:
sequence_df = utils.get_sequence(args=args, subject_dfs=subject_dfs, subject=4)
y = sequence_df['Pain'].values
image_paths = sequence_df['Path'].values
of_paths = sequence_df['OF_Path'].values

Chose subject  4
Start index in subject dataframe:  5918


In [9]:
sequence_df

,Unnamed: 0,Unnamed: 0.1,Video_ID,Path,Pain,Observer,Train,OF_Path
0,5918,5918,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000307.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002449.jpg
1,5919,5919,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000308.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002457.jpg
2,5920,5920,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000309.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002465.jpg
3,5921,5921,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000310.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002473.jpg
4,5922,5922,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000311.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002481.jpg
5,5923,5923,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000312.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002489.jpg
6,5924,5924,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000313.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002497.jpg
7,5925,5925,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000314.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002505.jpg
8,5926,5926,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000315.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002513.jpg
9,5927,5927,5_2b,data/jpg_128_128_2fps/horse_5/5_2b/frame_000316.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_5/5_2b/flow_002521.jpg


## Paths to the best models for each test subject

In [10]:
# @Maheen, I realized that I renamed these models somewhat in dropbox to remove
# the "CONVfilters_16" part of the name which doesn't apply, that was just a default setting,
# in case you attempt to load these.

t0 = '../models/BEST_MODEL_2stream_5d_adadelta_LSTMunits_32_CONVfilters_16_add_v4_t0_4hl_128jpg2fps_seq10_bs8_adadelta_noaug_run1_rerun_for_gradcam.h5'
t1 = '../models/BEST_MODEL_2stream_5d_add_adadelta_LSTMunits_32_CONVfilters_16_add_v4_t1_4hl_128jpg2fps_seq10_bs8_MAG_adadelta_noaug_run1_rerun_for_gradcam.h5'
t2 = '../models/BEST_MODEL_2stream_5d_add_adadelta_LSTMunits_32_CONVfilters_16_add_v4_t2_4hl_128jpg2fps_seq10_bs8_MAG_adadelta_noaug_run1_rerun_for_gradcam.h5'
t3 = '../models/BEST_MODEL_2stream_5d_add_adadelta_LSTMunits_32_CONVfilters_16_add_v4_t3_4hl_128jpg2fps_seq10_bs8_MAG_adadelta_noaug_run1_rerun_gc.h5'
t4 = '../models/BEST_MODEL_2stream_5d_add_adadelta_LSTMunits_32_CONVfilters_16_add_v0_t4_4hl_128jpg2fps_seq10_bs8_MAG_adadelta_noaug_run1_rerun_gc.h5'
t5 ='../models/BEST_MODEL_2stream_5d_add_adadelta_LSTMunits_32_CONVfilters_16_add_v4_t5_4hl_128jpg2fps_seq10_bs8_MAG_adadelta_noaug_run1_rerun_gc.h5'

In [11]:
model_t0 = keras.models.load_model('../models/BEST_MODEL_2stream_5d_adadelta_LSTMunits_32_CONVfilters_16_add_v4_t0_4hl_128jpg2fps_seq10_bs8_adadelta_noaug_run1_rerun_for_gradcam.h5')

In [12]:
model_t0.layers

In [13]:
model_t1 = keras.models.load_model('../models/BEST_MODEL_2stream_5d_add_adadelta_LSTMunits_32_CONVfilters_16_add_v0_t4_4hl_128jpg2fps_seq10_bs8_MAG_adadelta_noaug_run1_rerun_gc.h5')

In [14]:
model_t1.layers

In [15]:
model_t1.layers[2].layers

In [16]:
subject_dfs[0].head()

,Unnamed: 0,Unnamed: 0.1,Video_ID,Path,Pain,Observer,Train,OF_Path
0,0,0,1_1a,data/jpg_128_128_2fps/horse_1/1_1a_1/frame_000001.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_1/1_1a_1/flow_000001.jpg
1,1,1,1_1a,data/jpg_128_128_2fps/horse_1/1_1a_1/frame_000002.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_1/1_1a_1/flow_000009.jpg
2,2,2,1_1a,data/jpg_128_128_2fps/horse_1/1_1a_1/frame_000003.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_1/1_1a_1/flow_000017.jpg
3,3,3,1_1a,data/jpg_128_128_2fps/horse_1/1_1a_1/frame_000004.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_1/1_1a_1/flow_000025.jpg
4,4,4,1_1a,data/jpg_128_128_2fps/horse_1/1_1a_1/frame_000005.jpg,0,0,-1,data/jpg_128_128_16fps_OF_magnitude_cv2/horse_1/1_1a_1/flow_000033.jpg


In [17]:
def prepare_2stream_image_generator_5D(df, train, val, test, evaluate,
                                       rgb_period, flow_period):
    """
    Prepare the frames into labeled train and test sets, with help from the
    DataFrame with .jpg-paths and labels for train and pain.
    :param df: pd.DataFrame
    :param train: Boolean
    :param val: Boolean
    :param test: Boolean
    :param evaluate: Boolean
    :param rgb_period: int, if 10, take every 10th frame. # Not relevant here
    :param flow_period: int, if 1, take every frame.      # Not relevant here
    :return: np.ndarray, np.ndarray, np.ndarray, np.ndarray
    """

    nb_frames = len(df)
    print("LEN DF (nb. of frames): ", nb_frames)

    ws = args.seq_length  # "Window size" in a sliding window.
    ss = args.seq_stride  # Stride for the extracted windows
    valid = nb_frames - (ws - 1)
    nw = valid//ss  # Number of windows
    print('Number of windows', nw)

    this_index = 0
    seq_index = 0

    # Make sure that no augmented sequences are thrown away,
    # because we really want to augment the dataset.

    nb_aug = args.aug_flip + args.aug_crop + args.aug_light
    batch_requirement = 1 + nb_aug  # Normal sequence plus augmented sequences.
    assert (args.batch_size % batch_requirement) == 0

    while True:
        # Shuffle blocks between epochs.
        if train:
            df = shuffle_blocks(df, 'Video_ID')
        batch_index = 0
        for window_index in range(nw):
            start = window_index * ss
            stop = start + ws
            rows = df.iloc[start:stop]  # A new dataframe for the window in question.

            X_seq_list = []
            y_seq_list = []
            flow_seq_list = []

            for index, row in rows.iterrows():
                vid_seq_name = row['Video_ID']

                if this_index == 0:
                    print('First frame. Set oldname=vidname')
                    old_vid_seq_name = vid_seq_name  # This variable is set once
                    this_index += 1

                if vid_seq_name != old_vid_seq_name:
                    seq_index = 0
                    old_vid_seq_name = vid_seq_name
                    break  # Skip this one and jump to next window

                if (seq_index % rgb_period) == 0:
                    x = get_image(args, row['Path'])
                    X_seq_list.append(x)
                    y = row['Pain']
                    y_seq_list.append(y)

                if (seq_index % flow_period) == 0:
                    flow = get_image(args, row['OF_Path'])
                    if rgb_period > 1:
                        # We only want the first two channels of the flow.
                        flow = np.take(flow, [0,1], axis=2)
                    flow_seq_list.append(flow)

                seq_index += 1

            if batch_index == 0:
                X_batch_list = []
                y_batch_list = []
                flow_batch_list = []

            if seq_index == args.seq_length:
                if rgb_period > 1:
                    flow_seq_list = np.array(flow_seq_list)
                    flow_seq_list = np.reshape(np.array(flow_seq_list),
                                              (-1, args.image_size[0], args.image_size[1]))
                    X_seq_list = np.reshape(np.array(X_seq_list),
                                           (args.image_size[0], args.image_size[1], -1))

                X_batch_list.append(X_seq_list)
                y_batch_list.append(y_seq_list)
                flow_batch_list.append(flow_seq_list)
                batch_index += 1
                seq_index = 0

                if train and (args.aug_flip == 1):
                    # Flip both RGB and flow arrays
                    X_seq_list_flipped = args.flip_images(X_seq_list)
                    flow_seq_list_flipped = args.flip_images(flow_seq_list)
                    # Append to the respective batch lists
                    X_batch_list.append(X_seq_list_flipped)
                    y_batch_list.append(y_seq_list)
                    flow_batch_list.append(flow_seq_list_flipped)
                    batch_index += 1

                if train and (args.aug_crop == 1):
                    crop_size = 99
                    # Flip both RGB and flow arrays
                    X_seq_list_cropped = args.random_crop_resize(X_seq_list,
                                                                 crop_size, crop_size)
                    flow_seq_list_cropped = args.random_crop_resize(flow_seq_list,
                                                                    crop_size, crop_size)
                    # Append to the respective batch lists
                    X_batch_list.append(X_seq_list_cropped)
                    y_batch_list.append(y_seq_list)
                    flow_batch_list.append(flow_seq_list_cropped)
                    batch_index += 1

                if train and (args.aug_light == 1):
                    # Flip both RGB and flow arrays
                    X_seq_list_shaded = args.add_gaussian_noise(X_seq_list)
                    flow_seq_list_shaded = args.add_gaussian_noise(flow_seq_list)
                    # Append to the respective batch lists
                    X_batch_list.append(X_seq_list_shaded)
                    y_batch_list.append(y_seq_list)
                    flow_batch_list.append(flow_seq_list_shaded)
                    batch_index += 1

            if batch_index % args.batch_size == 0 and not batch_index == 0:
                X_array = np.array(X_batch_list, dtype=np.float32)
                y_array = np.array(y_batch_list, dtype=np.uint8)
                flow_array = np.array(flow_batch_list, dtype=np.float32)
                if args.nb_labels == 2:
                    y_array = np_utils.to_categorical(y_array, num_classes=args.nb_labels)
                    y_array = np.reshape(y_array, (args.batch_size, -1, args.nb_labels))
                else:
                    y_array = np.reshape(y_array, (args.batch_size, -1, args.nb_labels))
                if rgb_period > 1:
                    y_array = np.reshape(y_array, (args.batch_size, args.nb_labels))
                batch_index = 0
                # print(X_array.shape, flow_array.shape, y_array.shape)
                yield X_array, flow_array, y_array, start

In [18]:
def get_image(args, path):
    channels = 3 if dh.color else 1
    im = process_image('../' + path, (dh.image_size[0], dh.image_size[1], channels))
    return im

# Generate flattened features from both RGB and optical flow streams for each of the test subjects

## First subject (t0)

In [19]:
# First subject
df = subject_dfs[0]
model = keras.models.load_model(t0)

# Get intermediate output from the model (the flattened output of each separate stream)
get_last_rgb_output = K.function([model.layers[2].layers[0].input], [model.layers[2].layers[-1].output])
get_last_of_output = K.function([model.layers[3].layers[0].input], [model.layers[3].layers[-1].output])

features_t0 = []
counter = 0

for value in prepare_2stream_image_generator_5D(df,
                                                train=False,
                                                val=False,
                                                test=True,
                                                evaluate=True,
                                                rgb_period=1,
                                                flow_period=1):
    counter += 1
    if counter > 1431:
        break
    
    start_ind = value[3]
    
    print(counter, start_ind, end="\r")
    
    rgb = value[0]
    of = value[1]
    
    last_rgb_output = get_last_rgb_output([rgb])
    last_of_output = get_last_of_output([of])
    to_save = [last_rgb_output, last_of_output, start_ind]
    features_t0.append(to_save)


features_t0 = np.asarray(features_t0)
np.savez_compressed('../data/features_t0', features_t0=features_t0)

KeyboardInterrupt: 

## Second subject (t1)

In [145]:
# Second subject
df = subject_dfs[1]
model = keras.models.load_model(t1)

# Get intermediate output from the model (the flattened output of each separate stream)
get_last_rgb_output = K.function([model.layers[2].layers[0].input], [model.layers[2].layers[-1].output])
get_last_of_output = K.function([model.layers[3].layers[0].input], [model.layers[3].layers[-1].output])

features_t1 = []
counter = 0

for value in prepare_2stream_image_generator_5D(df,
                                                train=False,
                                                val=False,
                                                test=True,
                                                evaluate=True,
                                                rgb_period=1,
                                                flow_period=1):
    counter += 1
    if counter > 1210:
        break
    
    start_ind = value[3]
    
    print(counter, start_ind, end="\r")
    
    rgb = value[0]
    of = value[1]
    
    last_rgb_output = get_last_rgb_output([rgb])
    last_of_output = get_last_of_output([of])
    to_save = [last_rgb_output, last_of_output, start_ind]
    features_t1.append(to_save)


features_t1 = np.asarray(features_t1)
np.savez_compressed('../data/features_t1', features_t1=features_t1)

LEN DF (nb. of frames):  12118
Number of windows 1210
First frame. Set oldname=vidname


## Third subject (t2)

In [151]:
df = subject_dfs[2]
model = keras.models.load_model(t2)

# Get intermediate output from the model (the flattened output of each separate stream)
get_last_rgb_output = K.function([model.layers[2].layers[0].input], [model.layers[2].layers[-1].output])
get_last_of_output = K.function([model.layers[3].layers[0].input], [model.layers[3].layers[-1].output])

features_t2 = []
counter = 0

for value in prepare_2stream_image_generator_5D(df,
                                                train=False,
                                                val=False,
                                                test=True,
                                                evaluate=True,
                                                rgb_period=1,
                                                flow_period=1):
    counter += 1
    if counter > 1091:
        break
    
    start_ind = value[3]
    
    print(counter, start_ind, end="\r")
    
    rgb = value[0]
    of = value[1]
    
    last_rgb_output = get_last_rgb_output([rgb])
    last_of_output = get_last_of_output([of])
    to_save = [last_rgb_output, last_of_output, start_ind]
    features_t2.append(to_save)


features_t2 = np.asarray(features_t2)
np.savez_compressed('../data/features_t2', features_t2=features_t2)

LEN DF (nb. of frames):  10924
Number of windows 1091
First frame. Set oldname=vidname


## Fourth subject (t3)

In [20]:
df = subject_dfs[3]
model = keras.models.load_model(t3)

# Get intermediate output from the model (the flattened output of each separate stream)
get_last_rgb_output = K.function([model.layers[2].layers[0].input], [model.layers[2].layers[-1].output])
get_last_of_output = K.function([model.layers[3].layers[0].input], [model.layers[3].layers[-1].output])

features_t3 = []
counter = 0

for value in prepare_2stream_image_generator_5D(df,
                                                train=False,
                                                val=False,
                                                test=True,
                                                evaluate=True,
                                                rgb_period=1,
                                                flow_period=1):
    counter += 1
    if counter > 1361:
        break
    
    start_ind = value[3]
    
    print(counter, start_ind, end="\r")
    
    rgb = value[0]
    of = value[1]
    
    last_rgb_output = get_last_rgb_output([rgb])
    last_of_output = get_last_of_output([of])
    to_save = [last_rgb_output, last_of_output, start_ind]
    features_t3.append(to_save)


features_t3 = np.asarray(features_t3)
np.savez_compressed('../data/features_t3', features_t3=features_t3)

LEN DF (nb. of frames):  13626
Number of windows 1361
First frame. Set oldname=vidname


## Fifth subject (t4)

In [21]:
df = subject_dfs[4]
model = keras.models.load_model(t4)

# Get intermediate output from the model (the flattened output of each separate stream)
get_last_rgb_output = K.function([model.layers[2].layers[0].input], [model.layers[2].layers[-1].output])
get_last_of_output = K.function([model.layers[3].layers[0].input], [model.layers[3].layers[-1].output])

features_t4 = []
counter = 0

for value in prepare_2stream_image_generator_5D(df,
                                                train=False,
                                                val=False,
                                                test=True,
                                                evaluate=True,
                                                rgb_period=1,
                                                flow_period=1):
    counter += 1
    if counter > 1252:
        break
    
    start_ind = value[3]
    
    print(counter, start_ind, end="\r")
    
    rgb = value[0]
    of = value[1]
    
    last_rgb_output = get_last_rgb_output([rgb])
    last_of_output = get_last_of_output([of])
    to_save = [last_rgb_output, last_of_output, start_ind]
    features_t4.append(to_save)


features_t4 = np.asarray(features_t4)
np.savez_compressed('../data/features_t4', features_t4=features_t4)

LEN DF (nb. of frames):  12534
Number of windows 1252
First frame. Set oldname=vidname


## Sixth subject (t5)

In [22]:
df = subject_dfs[5]
model = keras.models.load_model(t5)

# Get intermediate output from the model (the flattened output of each separate stream)
get_last_rgb_output = K.function([model.layers[2].layers[0].input], [model.layers[2].layers[-1].output])
get_last_of_output = K.function([model.layers[3].layers[0].input], [model.layers[3].layers[-1].output])

features_t5 = []
counter = 0

for value in prepare_2stream_image_generator_5D(df,
                                                train=False,
                                                val=False,
                                                test=True,
                                                evaluate=True,
                                                rgb_period=1,
                                                flow_period=1):
    counter += 1
    if counter > 668:
        break
    
    start_ind = value[3]
    
    print(counter, start_ind, end="\r")
    
    rgb = value[0]
    of = value[1]
    
    last_rgb_output = get_last_rgb_output([rgb])
    last_of_output = get_last_of_output([of])
    to_save = [last_rgb_output, last_of_output, start_ind]
    features_t5.append(to_save)


features_t5 = np.asarray(features_t5)
np.savez_compressed('../data/features_t5', features_t5=features_t5)

LEN DF (nb. of frames):  6692
Number of windows 668
First frame. Set oldname=vidname


In [141]:
fs = np.load('../data/features_t0.npz')
print(fs['features_t0'][0])

[list([array([[[-0.16999894, -0.13054055,  0.23271295, ...,  0.17906334,
         -0.3020972 , -0.80027014],
        [ 0.14207786, -0.14978519,  0.3541257 , ...,  0.3062452 ,
         -0.19731688, -0.7591243 ],
        [ 0.5062062 , -0.09573984,  0.3479384 , ...,  0.41385764,
         -0.29754314, -0.5495806 ],
        ...,
        [ 1.5066361 ,  0.4606979 ,  0.23010904, ...,  0.3948507 ,
         -0.46377826,  0.19836313],
        [ 1.5685604 ,  0.80339056,  0.19359308, ...,  0.3757646 ,
         -0.21516004,  0.07680392],
        [ 1.5875493 ,  1.1781039 ,  0.14046088, ...,  0.50268376,
         -0.02485543, -0.03744203]]], dtype=float32)])
 list([array([[[-0.24134095,  0.2715644 ,  0.34875235, ..., -0.2629024 ,
         -0.84345174, -0.42142418],
        [-0.204056  ,  0.13848266,  0.4299632 , ...,  0.01679349,
         -0.770345  , -0.28449717],
        [-0.0386588 ,  0.00661561,  0.5900829 , ...,  0.27658594,
         -0.7818452 , -0.08397353],
        ...,
        [ 0.7711395 ,  